In [1]:
import glob
import os

import pandas as pd
import polars as pl
import polars.selectors as cs

In [2]:
labels = pl.concat([
    pl.read_csv("/home/generic-user/Downloads/windmark-main/data/raw/validation.csv.gz"),
    pl.read_csv("/home/generic-user/Downloads/windmark-main/data/raw/training.csv.gz"),
    pl.read_csv("/home/generic-user/Downloads/windmark-main/data/raw/holdout.csv.gz").select(
        "original_loan_id",
        pl.lit(None).alias("boolean_bad"),
        "open_mth_original",
        pl.col("dataset").replace("holdout", "test")
    )
    
])

labels = labels.to_pandas().set_index("original_loan_id")

In [3]:
dfs = []

file_index = 0

for x in os.walk("/home/generic-user/Downloads/windmark-main/data/raw/tradeline_data_extract/s3"):
    
    for y in glob.glob(os.path.join(x[0], '*.json.gz')):

        file_index += 1

        df = (
            pl.from_pandas(
                pd.read_json(y, lines=True).set_index("loan_account_ref").join(labels, how="left")
            )
            .select(
                pl.exclude("dataset", "boolean_bad"),
                pl.col("dataset").replace(None, "test").replace("training", "train").replace("validation", "validate"),
                is_bad=pl.col("boolean_bad").cast(pl.String),
            )
        )

        dynamic = (
            df.select(
                "bureau_search_ref",
                pl.col("created_at").str.replace("\+00", "").str.to_date(format="%Y-%m-%d %H:%M:%S%.f"),
                cs.by_dtype(pl.List(pl.String)) | cs.by_dtype(pl.List(pl.Int64)),
                target=pl.col("is_bad"),
            )
            .explode(pl.exclude("created_at","bureau_search_ref", "target"))
            .select(
                cs.all() - cs.string() - cs.by_name("account_index"),
                (cs.string() - cs.by_name("m")).replace("", None).replace("{ND}", None).replace("null", None),
                statement_id=pl.concat_str("accgroupid", pl.lit("$"), "m"),
                account_open=pl.col("accstartdate").str.to_date(),
                account_index=pl.col("accgroupid").cast(pl.String),
                statement_date=pl.col("m").str.to_date(format="%Y-%m"),
            )
            .select(
                "payamt", "bal", "history_limit", "bureau_search_ref", "pay", "acctypecode", "target", "statement_id", "account_index",
                # calculate two duration fields (time diff between statement and application date, statement and account open date)
                statement_tenure=(pl.col("statement_date") - pl.col("created_at")).dt.total_days(),
                account_tenure=(pl.col("account_open") - pl.col("created_at")).dt.total_days(),
            )
            .sort("bureau_search_ref", "statement_tenure")
            .group_by("bureau_search_ref", maintain_order=True)
            .agg(cs.all())
        )

        static = (
            df
            .select(
                cs.string().replace("", None).replace("{ND}", None).replace("null", None),
                cs.numeric(),
            )
            .select(
                pl.exclude("nob", "urb"),
                pl.col("nob").cast(pl.String),
                pl.col("urb").cast(pl.String),
            )
        )

        ledger = dynamic.join(static, on='bureau_search_ref')

        for shard_index, shard in enumerate(ledger.iter_slices(20)):
            shard.write_ndjson(f"/home/generic-user/Downloads/windmark-main/data/lifestreams/plata/plata-{file_index}-{shard_index}.ndjson")

In [4]:
dynamic.select(pl.col('target').list.first(), pl.col('target').list.last().alias('last'))

target,last
str,str
"""false""","""false"""
"""false""","""false"""
"""false""","""false"""
"""false""","""false"""
"""true""","""true"""
…,…
"""true""","""true"""
"""false""","""false"""
"""false""","""false"""


In [5]:
df = pl.read_avro('/home/generic-user/Downloads/windmark-main/data/lifestreams/plata/plata-2-2.avro')

FileNotFoundError: No such file or directory (os error 2): /home/generic-user/Downloads/windmark-main/data/lifestreams/plata/plata-2-2.avro

In [ ]:
df

bureau_search_ref,payamt,bal,history_limit,pay,acctypecode,target,statement_id,account_index,statement_tenure,account_tenure,oq,ubc,cbc,bf,nq,irb,hrb,oyb,ibc,open_mth_original,dataset,is_bad,ygc,nic,ueb,qic,reb,pic,qeb,oic,teb,seb,eb,zd,jyb,ye,…,yeb,total_balance_on_active_account1,total_balance_on_default_accoun4,xzb,tp,ccc,tr,gg,ah,as,lrb,wm,pob,vyb,ef,vm,rh,tic,xe,wmb,fs,gauge2_score,fb,mr,total_repayments_on_active_acco5,pn,azb,ej,mhc,hn,rrb,kn,jg,total_repayments_on_active_acco2,izb,nob,urb
str,list[i64],list[i64],list[i64],list[str],list[str],list[str],list[str],list[str],list[i64],list[i64],str,str,str,str,str,str,str,str,str,str,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,str,str
"""BS86QXB""","[null, null, … null]","[128130, 128134, … 0]","[null, null, … 0]","[""0"", ""0"", … ""0""]","[""MG"", ""MG"", … ""EE""]","[""false"", ""false"", … ""false""]","[""2$2013-01"", ""2$2013-02"", … ""5$2017-09""]","[""2"", ""2"", … ""5""]","[-2163, -2132, … -459]","[-4793, -4793, … -533]","""U""","""U""","""U""","""B""","""U""","""U""","""U""",null,null,"""2018-12-31""","""train""","""true""",1.0,1.0,40.0,6.0,11.0,5.0,6.0,3.0,40.0,19.0,17.0,null,1.0,170.0,…,0.0,5436,0,36.0,0.0,4.0,13.0,69.0,3.0,0.0,0.0,112.0,101.0,null,4.0,102.0,8.0,109.0,58.0,12.0,null,525.0,12.0,1.0,0,691.0,0.0,3.0,5.0,208.0,239.0,107.0,9274.0,0,0.0,"""07""","""3"""
"""BS87746""","[null, null, … null]","[0, 90, … 0]","[2500, 0, … 0]","[""0"", ""0"", … ""0""]","[""CA"", ""EE"", … ""CA""]","[""false"", ""false"", … ""false""]","[""7$2017-09"", ""5$2017-10"", … ""7$2016-09""]","[""7"", ""5"", … ""7""]","[-459, -429, … -821]","[-565, -533, … -3141]","""0""","""0""","""U""","""G""","""0""","""0""","""0""",null,null,"""2018-12-31""","""train""","""true""",0.0,0.0,3.0,0.0,2.0,0.0,2.0,0.0,3.0,2.0,10.0,null,7.0,null,…,0.0,9247,0,4.0,0.0,6.0,7.0,76.0,3.0,null,2.0,97.0,1200.0,144.0,8.0,97.0,7.0,93.0,null,12.0,null,586.0,7.0,1.0,0,null,19.0,0.0,2.0,1297.0,null,93.0,9731.0,0,50.0,"""07""","""3"""
"""BS82J6Z""","[null, null, … null]","[0, 0, … 0]","[null, null, … 0]","[""0"", ""0"", … ""0""]","[""UL"", ""TM"", … ""HC""]","[""false"", ""false"", … ""true""]","[""1$2016-09"", ""4$2016-10"", … ""10$2014-09""]","[""1"", ""4"", … ""10""]","[-821, -791, … -1562]","[-1683, -2406, … -2076]","""U""","""6""","""D""","""D""","""0""","""D""","""D""",null,null,"""2018-12-31""","""train""","""false""",0.0,0.0,4.0,1.0,3.0,1.0,1.0,1.0,4.0,4.0,28.0,null,33.0,122.0,…,0.0,1099,104,26.0,0.0,2.0,18.0,97.0,0.0,581.0,0.0,121.0,62.0,null,4.0,94.0,9.0,73.0,98.0,13.0,643.0,549.0,15.0,2.0,0,828.0,0.0,6.0,3.0,135.0,null,97.0,4025.0,0,null,"""21""","""6"""
"""BS82HW3""","[null, null, … 0]","[128, 0, … 426]","[0, 0, … 500]","[""D"", ""0"", … ""0""]","[""CA"", ""CA"", … ""CC""]","[""true"", ""true"", … ""false""]","[""7$2014-10"", ""7$2014-10"", … ""3$2014-03""]","[""7"", ""7"", … ""3""]","[-1532, -1532, … -1736]","[-4298, -4637, … -1902]","""U""","""U""","""0""","""A""","""0""","""U""","""U""",null,null,"""2018-12-31""","""train""","""true""",0.0,0.0,11.0,0.0,3.0,0.0,2.0,0.0,11.0,8.0,75.0,null,1.0,47.0,…,0.0,8621,0,48.0,0.0,null,null,75.0,1.0,null,19.0,101.0,60.0,109.0,5.0,102.0,11.0,81.0,47.0,83.0,null,565.0,11.0,0.0,0,8898.0,0.0,0.0,7.0,192.0,200.0,246.0,16610.0,0,0.0,"""22""","""7"""
"""BS86TH4""","[null, null, … 120]","[0, 0, … 1351]","[0, null, … 1400]","[""U"", ""0"", … ""0""]","[""CA"", ""TM"", … ""CC""]","[""false"", ""false"", … ""false""]","[""7$2014-03"", ""4$2014-03"", … ""3$2018-09""]","[""7"", ""4"", … ""3""]","[-1736, -1736, … -91]","[-2585, -1853, … -1520]","""0""","""0""","""U""","""B""","""0""","""0""","""0""",null,"""0""","""2018-12-31""","""train""","""false""",0.0,0.0,6.0,1.0,1.0,0.0,1.0,0.0,4.0,2.0,25.0,null,14.0,182.0,…,0.0,17681,0,24.0,0.0,null,null,160.0,2.0,null,12.0,106.0,87.0,40.0,2.0,99.0,11.0,null,182.0,47.0,null,615.0,

In [ ]:
df.select('statement_id', 'account_tenure').head(1).to_dict(as_series=False)

{'statement_id': [['2$2013-01',
   '2$2013-02',
   '2$2013-03',
   '2$2013-04',
   '2$2013-05',
   '2$2013-06',
   '2$2013-07',
   '2$2013-08',
   '2$2013-09',
   '2$2013-10',
   '2$2013-11',
   '2$2013-12',
   '2$2014-01',
   '2$2014-02',
   '2$2014-03',
   '2$2014-04',
   '2$2014-05',
   '2$2014-06',
   '2$2014-07',
   '2$2014-08',
   '2$2014-09',
   '2$2014-10',
   '2$2014-11',
   '2$2014-12',
   '2$2015-01',
   '2$2015-02',
   '2$2015-03',
   '2$2015-04',
   '2$2015-05',
   '2$2015-06',
   '3$2015-06',
   '3$2015-07',
   '2$2015-07',
   '2$2015-08',
   '3$2015-08',
   '2$2015-09',
   '3$2015-09',
   '3$2015-10',
   '2$2015-10',
   '2$2015-11',
   '3$2015-11',
   '1$2015-11',
   '3$2015-12',
   '1$2015-12',
   '2$2015-12',
   '3$2016-01',
   '1$2016-01',
   '2$2016-01',
   '1$2016-02',
   '2$2016-02',
   '3$2016-02',
   '3$2016-03',
   '2$2016-03',
   '1$2016-03',
   '2$2016-04',
   '3$2016-04',
   '1$2016-04',
   '3$2016-05',
   '2$2016-05',
   '1$2016-05',
   '2$2016-06',
   '3$20

In [ ]:
# dfs = []

# file_index = 0

# for x in os.walk("/home/generic-user/Downloads/windmark-main/data/raw/tradeline_data_extract/s3"):
    
#     for y in glob.glob(os.path.join(x[0], '*.json.gz')):

#         file_index += 1

#         df = (
#             pl.from_pandas(
#                 pd.read_json(y, lines=True).set_index("loan_account_ref").join(labels, how="left")
#             )
#             .select(
#                 pl.exclude("dataset", "boolean_bad"),
#                 pl.col("dataset").replace(None, "test").replace("training", "train").replace("validation", "validate"),
#                 is_bad=pl.col("boolean_bad").cast(pl.String),
#             )
#         )

#         dynamic = (
#             df
#             .select(
#                 pl.all(),
#                 size=pl.col('account_index').list.len()
#             )
#             .select(
#                 cs.by_dtype(pl.List(pl.String)) | cs.by_dtype(pl.List(pl.Int64)),
#                 target=pl.col("is_bad").repeat_by('size'),
#                 bureau_search_ref=pl.col("bureau_search_ref"),
#             )
#             .select(
#                 'target',
#                 'bureau_search_ref',
#                 'payamt',
#                 'acctypecode',
#                 'pay',
#                 'bal',
#                 pl.col("account_index").list.eval(pl.element().cast(pl.String)),
#                 statement_id=pl.col('bal')
#             )
#         )

#         static = (
#             df
#             .select(
#                 cs.string().replace("", None).replace("{ND}", None).replace("null", None),
#                 cs.numeric(),
#             )
#             .select(
#                 pl.exclude("nob", "urb"),
#                 pl.col("nob").cast(pl.String),
#                 pl.col("urb").cast(pl.String),
#             )
#         )

#         ledger = dynamic.join(static, on='bureau_search_ref')

#         for shard_index, shard in enumerate(ledger.iter_slices(20)):
#             shard.write_ndjson(f"/home/generic-user/Downloads/windmark-main/data/lifestreams/plata/plata-{file_index}-{shard_index}.ndjson")